In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = True
ROUNDS = 50

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

md = md[0:1]
print("shape:", md.shape)
# md[18:]
md

state = X0
shape: (1, 18)


,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info
5086,2024-03-23 14:11:49.436000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzda58czq6g0081he8g,"[Subset 52, 50 rounds, 0 log, xbasis=True]",1.0,1507.0,NaN,JobStatus.DONE,2024-03-23 15:41:07.958655+01:00,RepetitionCodeCircuit,52,50,0,subset RepCodes,NaN,True,"{'mean_gate_error': 0.007733045348037, 'min_ga..."


# Group job ids by closest calibration date

In [4]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [5]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 13, 12, 1, 375000, tzinfo=datetime.timezone.utc): ['cqzda58czq6g0081he8g']}

num of calibrations: 1
num of jobs per calibration: [1]


# Decode data

In [6]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]

distances

array([51, 47, 43, 39, 35, 31, 27, 23, 19, 15, 11,  7])

In [7]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi
from soft_info import gaussianKDEIQconvertor, get_KDEs

file_name  = f'./results/result_day2/GAUSSIANKDE{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
# lin = [0.1, 0.7, 1]
# num_points = 7
bandwidths = np.linspace(lin[0], lin[1], lin[2])

bandwidth = 0.1 # Gaussians
# bandwidths = 0.2 # Tophat


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)
    print(f"\nQubit 20 shape mmr0: {all_memories[20]['mmr_0'].shape}\n")

    # kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)
    kde_dict, scaler_dict = get_KDEs(all_memories, bandwidths = bandwidth)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)

    print(f"\nQubit 20 for PS shape mmr0: {all_memories_PS[20]['mmr_0'].shape}\n")
    # kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)
    kde_dict_PS, scaler_dict_PS = get_KDEs(all_memories_PS, bandwidths = bandwidth)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    # print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

    # for i in range(127):
    #     print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
    # print("\nPS", end=' ')
    # for i in range(127):
    #     print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # big_memory = big_memory[:NB_SHOTS//100] # 1% of the data
    print(f"big_memory shape: {big_memory.shape}")

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    # print(f"Starting to get pSoft and countMat at {datetime.now()}")
    # pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
    #                                             handleOutliers = HANDLE_OUTLIERS)
    
    # print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
    # pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
    #                                                   handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoftG at {datetime.now()}")
    countMatG, pSoftG = gaussianIQConvertor(big_memory, inverted_q_map, gmm_dict)

    print(f"Starting to get pSoftGKDE at {datetime.now()}")
    countMatGKDE, pSoftGKDE = gaussianKDEIQconvertor(big_memory, inverted_q_map, kde_dict, scaler_dict)
    print(f"Finished to get pSoftGKDE at {datetime.now()}")

    print(f"Starting to get pSoftGKDE_PS at {datetime.now()}")
    countMatGKDE_PS, pSoftGKDE_PS = gaussianKDEIQconvertor(big_memory, inverted_q_map, kde_dict_PS, scaler_dict_PS)
    print(f"Finished to get pSoftGKDE_PS at {datetime.now()}")


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_G_mean = []
        err_s_G_indiv = []
        err_h_G_indiv = []
        err_h_G_mean = []
        # err_s_K_mean = []
        # err_s_K_indiv = []
        # err_h_K_indiv = []
        # err_h_K_mean = []
        # err_s_KPS_mean = []
        # err_s_KPS_indiv = []
        # err_h_KPS_indiv = []
        # err_h_KPS_mean = []
        err_h_G_mean_mean = []
        # err_h_K_mean_mean = []
        # err_h_KPS_mean_mean = []
        err_s_GKDE_mean = []
        err_s_GKDE_indiv = []
        err_h_GKDE_indiv = []
        err_h_GKDE_mean = []
        err_h_GKDE_mean_mean = []

        err_s_GKDE_PS_mean = []
        err_s_GKDE_PS_indiv = []
        err_h_GKDE_PS_indiv = []
        err_h_GKDE_PS_mean = []
        err_h_GKDE_PS_mean_mean = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            # pSoft_sub = pSoft[:, cols_to_keep]
            # pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
            pSoftG_sub = pSoftG[:, cols_to_keep]
            # countMat_sub = countMat[:, cols_to_keep]
            # countMat_PS_sub = countMat_PS[:, cols_to_keep]
            countMatG_sub = countMatG[:, cols_to_keep]
            countMatGKDE_sub = countMatGKDE[:, cols_to_keep]
            pSoftGKDE_sub = pSoftGKDE[:, cols_to_keep]
            countMatGKDE_PS_sub = countMatGKDE_PS[:, cols_to_keep]
            pSoftGKDE_PS_sub = pSoftGKDE_PS[:, cols_to_keep]

            # Means for hard decoding
            # pSoft_mean = np.mean(pSoft_sub)
            # pSoft_PS_mean = np.mean(pSoft_PS_sub)
            pSoftG_mean = np.mean(pSoftG_sub)
            pSoftGKDE_mean = np.mean(pSoftGKDE_sub)
            pSoftGKDE_PS_mean = np.mean(pSoftGKDE_PS_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
            

            # Decoding

            ### Gaussian KDE
            res_s_GKDE_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatGKDE_sub, pSoftGKDE_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_GKDE_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatGKDE_sub, pSoftGKDE_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_GKDE_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatGKDE_sub, pSoftGKDE_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_GKDE_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatGKDE_sub, pSoftGKDE_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftGKDE_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_GKDE_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatGKDE_sub, pSoftGKDE_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)

            ### Gaussian KDE PS
            res_s_GKDE_PS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatGKDE_PS_sub, pSoftGKDE_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_GKDE_PS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatGKDE_PS_sub, pSoftGKDE_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_GKDE_PS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatGKDE_PS_sub, pSoftGKDE_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_GKDE_PS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatGKDE_PS_sub, pSoftGKDE_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftGKDE_PS_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_GKDE_PS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatGKDE_PS_sub, pSoftGKDE_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)

            ### Gaussian
            res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                  int(LOGICAL), _RESETS, decode_hard=False)
            res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftG_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            # ### KDE
            # res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=False)
            # res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=False)
            # res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=True)
            # res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=True)
            
            # noise_list[-1] = pSoft_mean
            # code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
            #                                             subsampling=subsampling, no_fin_soft=False, layout=None,
            #                                             msmt_err_dict=None)
            # model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            # res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
            #                                             int(LOGICAL), _RESETS, decode_hard=True)
            
            # ### KDE PS
            # res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=False)
            # res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=False)
            # res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=True)
            # res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
            #                                         int(LOGICAL), _RESETS, decode_hard=True)
            
            # noise_list[-1] = pSoft_PS_mean
            # code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
            #                                             subsampling=subsampling, no_fin_soft=False, layout=None,
            #                                             msmt_err_dict=None)
            # model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            # res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
            #                                             int(LOGICAL), _RESETS, decode_hard=True)
            
            # Append the results
            err_s_GKDE_mean.append(res_s_GKDE_mean.num_errors)
            err_s_GKDE_indiv.append(res_s_GKDE_indiv.num_errors)
            err_h_GKDE_indiv.append(res_h_GKDE_indiv.num_errors)
            err_h_GKDE_mean.append(res_h_GKDE_mean.num_errors)

            err_h_GKDE_mean_mean.append(res_h_GKDE_mean_mean.num_errors)

            err_s_GKDE_PS_mean.append(res_s_GKDE_PS_mean.num_errors)
            err_s_GKDE_PS_indiv.append(res_s_GKDE_PS_indiv.num_errors)
            err_h_GKDE_PS_indiv.append(res_h_GKDE_PS_indiv.num_errors)
            err_h_GKDE_PS_mean.append(res_h_GKDE_PS_mean.num_errors)

            err_h_GKDE_PS_mean_mean.append(res_h_GKDE_PS_mean_mean.num_errors)


            err_s_G_mean.append(res_s_G_mean.num_errors)
            err_s_G_indiv.append(res_s_G_indiv.num_errors)
            err_h_G_indiv.append(res_h_G_indiv.num_errors)
            err_h_G_mean.append(res_h_G_mean.num_errors)
            # err_s_K_mean.append(res_s_K_mean.num_errors)
            # err_s_K_indiv.append(res_s_K_indiv.num_errors)
            # err_h_K_indiv.append(res_h_K_indiv.num_errors)
            # err_h_K_mean.append(res_h_K_mean.num_errors)
            # err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
            # err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
            # err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
            # err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

            err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
            # err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
            # err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)


        # Save the results

        json_GKDE = {
            "decoding": "GaussianKDE",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_GKDE_mean,
                "soft_indiv": err_s_GKDE_indiv,
                "hard_indiv": err_h_GKDE_indiv,
                "hard_mean": err_h_GKDE_mean,
                "hard_mean_mean": err_h_GKDE_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_GKDE_mean),
                "soft_indiv": np.mean(err_s_GKDE_indiv),
                "hard_indiv": np.mean(err_h_GKDE_indiv),
                "hard_mean": np.mean(err_h_GKDE_mean),
                "hard_mean_mean": np.mean(err_h_GKDE_mean_mean),
                "pSoftGKDE_mean": pSoftGKDE_mean,
            },
            "additional_info": {
                "calib_date": str(calib_date),
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softGKDE_mean": pSoftGKDE_mean,
            },
        }

        json_GKDE_PS = {
            "decoding": "GaussianKDE_PS",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_GKDE_PS_mean,
                "soft_indiv": err_s_GKDE_PS_indiv,
                "hard_indiv": err_h_GKDE_PS_indiv,
                "hard_mean": err_h_GKDE_PS_mean,
                "hard_mean_mean": err_h_GKDE_PS_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_GKDE_PS_mean),
                "soft_indiv": np.mean(err_s_GKDE_PS_indiv),
                "hard_indiv": np.mean(err_h_GKDE_PS_indiv),
                "hard_mean": np.mean(err_h_GKDE_PS_mean),
                "hard_mean_mean": np.mean(err_h_GKDE_PS_mean_mean),
            },
            "additional_info": {
                "p_softGKDE_PS_mean": pSoftGKDE_PS_mean,
            },
        }

        json_Gaussian = {
            "decoding": "1Dgaussian",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_G_mean,
                "soft_indiv": err_s_G_indiv,
                "hard_indiv": err_h_G_indiv,
                "hard_mean": err_h_G_mean,
                "hard_mean_mean": err_h_G_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_G_mean),
                "soft_indiv": np.mean(err_s_G_indiv),
                "hard_indiv": np.mean(err_h_G_indiv),
                "hard_mean": np.mean(err_h_G_mean),
                "hard_mean_mean": np.mean(err_h_G_mean_mean),
            },      
            "additional_info": {
                "calib_date": str(calib_date),
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softG_mean": pSoftG_mean,
            },    
        }

        # json_KDE = {
        #     "decoding": "kde",
        #     "d_new": str(D_NEW),
        #     "error_list_dict": {
        #         "soft_mean": err_s_K_mean,
        #         "soft_indiv": err_s_K_indiv,
        #         "hard_indiv": err_h_K_indiv,
        #         "hard_mean": err_h_K_mean,
        #         "hard_mean_mean": err_h_K_mean_mean,
        #     },
        #     "error_mean_dict": {
        #         "soft_mean": np.mean(err_s_K_mean),
        #         "soft_indiv": np.mean(err_s_K_indiv),
        #         "hard_indiv": np.mean(err_h_K_indiv),
        #         "hard_mean": np.mean(err_h_K_mean),
        #         "hard_mean_mean": np.mean(err_h_K_mean_mean),
        #     },   
        #     "additional_info": {
        #         "rel_error": rel_error,
        #         "bandwidth_linspace": lin,
        #         "num_points_bandwidths": num_points,
        #         "pSoft_mean": pSoft_mean,
        #     },        
        # }

        # json_KDE_PS = {
        #     "decoding": "kde_PS",
        #     "d_new": str(D_NEW),
        #     "error_list_dict": {
        #         "soft_mean": err_s_KPS_mean,
        #         "soft_indiv": err_s_KPS_indiv,
        #         "hard_indiv": err_h_KPS_indiv,
        #         "hard_mean": err_h_KPS_mean,
        #         "hard_mean_mean": err_h_KPS_mean_mean,
        #     },
        #     "error_mean_dict": {
        #         "soft_mean": np.mean(err_s_KPS_mean),
        #         "soft_indiv": np.mean(err_s_KPS_indiv),
        #         "hard_indiv": np.mean(err_h_KPS_indiv),
        #         "hard_mean": np.mean(err_h_KPS_mean),
        #         "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
        #         "pSoft_PS_mean": pSoft_PS_mean,
        #     },   
        # }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_GKDE, json_GKDE_PS]
            # data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
        else:
            data[job_id].append(json_Gaussian)
            data[job_id].append(json_GKDE)
            data[job_id].append(json_GKDE_PS)
            # data[job_id].append(json_KDE)
            # data[job_id].append(json_KDE_PS)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

13:57:03 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
13:57:03 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
13:57:03 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.

Qubit 20 shape mmr0: (31496,)

Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:12:25.263713+00:00.

Qubit 20 for PS shape mmr0: (9078,)



Retrieving jobs of 2024-03-23 13:12:01.375000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.98s/it]


big_memory shape: (1507, 2602)
Starting to get pSoftG at 2024-04-01 13:58:03.936511
Starting to get pSoftGKDE at 2024-04-01 13:58:04.489853


100%|██████████| 103/103 [1:33:03<00:00, 54.21s/it]  


Finished to get pSoftGKDE at 2024-04-01 15:31:08.203429
Starting to get pSoftGKDE_PS at 2024-04-01 15:31:08.203445


100%|██████████| 103/103 [43:11<00:00, 25.16s/it] 


Finished to get pSoftGKDE_PS at 2024-04-01 16:14:19.918078


100%|██████████| 12/12 [03:53<00:00, 19.42s/it]
